In [13]:
# ==============================================================================
# @title 1. Instalasi dan Impor Library
# ==============================================================================
# Jalankan sel ini terlebih dahulu untuk mengimpor semua library yang dibutuhkan.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import os
import joblib
import warnings

warnings.filterwarnings('ignore')
print("✅ Library berhasil diimpor.")

✅ Library berhasil diimpor.


In [14]:
# ==============================================================================
# @title 2. Konfigurasi Utama
# ==============================================================================
# Sel ini mendefinisikan variabel-variabel penting seperti lokasi folder
# dan daftar kolom yang akan digunakan dalam model.

# Tentukan path folder sumber data dan folder untuk menyimpan hasil
SOURCE_DATA_DIR = 'sumber_data'
RESULTS_DIR = 'hasil_model'

# Pastikan folder hasil utama dan folder sumber ada
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(SOURCE_DATA_DIR, exist_ok=True)

# Daftar kolom fitur yang akan digunakan (tanpa 'Apparent Temperature')
RELEVANT_COLUMNS = [
    'Konsumsi Energi', 'Temperature', 'Showers', 'Cloud Cover', 'Weather Code',
    'Relative Humidity', 'Dew Point', 'Precipitation',
    'Pressure MSL', 'Surface Pressure', 'Evapotranspiration',
    'Vapour Pressure Deficit', 'Wind Speed', 'Wind Direction', 'Wind Gusts',
    'Soil Temperature', 'Sunshine Duration', 'UV Index', 'Direct Radiation'
]
TARGET_VARIABLE = 'Konsumsi Energi'

print(f"📁 Folder sumber data diatur ke: '{SOURCE_DATA_DIR}'")
print(f"📁 Folder hasil akan disimpan di: '{RESULTS_DIR}'")

📁 Folder sumber data diatur ke: 'sumber_data'
📁 Folder hasil akan disimpan di: 'hasil_model'


In [15]:
# ==============================================================================
# @title 3. Persiapan Folder dan Unggah Data
# ==============================================================================
# PENTING: Sebelum menjalankan sel-sel berikutnya, unggah data Anda.
#
# 1. Di panel file sebelah kiri Google Colab, Anda akan melihat folder 'sumber_data'.
# 2. Klik kanan pada folder 'sumber_data' tersebut dan pilih 'Upload'.
# 3. Unggah folder 'witel' dan 'opmc' Anda yang berisi semua data CSV
#    ke dalam folder 'sumber_data'.
#
# Setelah selesai, Anda bisa melanjutkan menjalankan sel-sel berikutnya.

print("✅ Sel ini siap.")
print(f"Pastikan Anda telah mengunggah data Anda ke dalam folder '{SOURCE_DATA_DIR}'.")


✅ Sel ini siap.
Pastikan Anda telah mengunggah data Anda ke dalam folder 'sumber_data'.


In [16]:
# ==============================================================================
# @title 4. Definisi Fungsi-Fungsi Pembantu
# ==============================================================================
# Sel ini berisi fungsi-fungsi utama untuk melatih model dan membuat visualisasi.
# Jalankan sel ini untuk mendefinisikan fungsi agar bisa digunakan nanti.

def train_and_evaluate_models(X_train, y_train, X_val, y_val, X_test, y_test):
    """
    Fungsi untuk melatih semua model (RF, GB, LSTM) dan mengembalikan
    prediksi serta metrik kinerjanya.
    """
    results = {}

    # --- Model 1: Random Forest Regressor ---
    print("   - Melatih Random Forest...")
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_test)
    results['RandomForest'] = {'model': rf_model, 'predictions': y_pred_rf, 'rmse': np.sqrt(mean_squared_error(y_test, y_pred_rf)), 'r2': r2_score(y_test, y_pred_rf)}

    # --- Model 2: Gradient Boosting Regressor ---
    print("   - Melatih Gradient Boosting...")
    gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    gb_model.fit(X_train, y_train)
    y_pred_gb = gb_model.predict(X_test)
    results['GradientBoosting'] = {'model': gb_model, 'predictions': y_pred_gb, 'rmse': np.sqrt(mean_squared_error(y_test, y_pred_gb)), 'r2': r2_score(y_test, y_pred_gb)}

    # --- Model 3: LSTM ---
    print("   - Melatih LSTM...")
    scaler_X = MinMaxScaler(feature_range=(0, 1)); scaler_y = MinMaxScaler(feature_range=(0, 1))
    X_train_scaled = scaler_X.fit_transform(X_train); y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
    X_val_scaled = scaler_X.transform(X_val); y_val_scaled = scaler_y.transform(y_val.values.reshape(-1, 1))
    X_test_scaled = scaler_X.transform(X_test)
    X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
    X_val_lstm = X_val_scaled.reshape((X_val_scaled.shape[0], 1, X_val_scaled.shape[1]))
    X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))
    lstm_model = Sequential([LSTM(50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])), Dense(1)])
    lstm_model.compile(optimizer='adam', loss='mean_squared_error')
    lstm_model.fit(X_train_lstm, y_train_scaled, epochs=50, batch_size=32, validation_data=(X_val_lstm, y_val_scaled), verbose=0, shuffle=False)
    y_pred_lstm_scaled = lstm_model.predict(X_test_lstm)
    y_pred_lstm = scaler_y.inverse_transform(y_pred_lstm_scaled)
    results['LSTM'] = {'model': lstm_model, 'predictions': y_pred_lstm.flatten(), 'rmse': np.sqrt(mean_squared_error(y_test, y_pred_lstm)), 'r2': r2_score(y_test, y_pred_lstm)}
    return results

def create_prediction_plots(y_test, predictions, plot_suffix, output_dir):
    """Membuat dan menyimpan scatter plot dan line graph untuk prediksi."""
    # --- Konversi ke kWh untuk semua plot ---
    y_test_kwh = y_test / 1000
    predictions_kwh = {name: pred / 1000 for name, pred in predictions.items()}

    # --- Scatter Plot ---
    plt.figure(figsize=(18, 6))
    colors = ['green', 'red', 'orange']
    for i, (model_name, pred_kwh) in enumerate(predictions_kwh.items()):
        # Metrik dihitung dalam kWh juga untuk konsistensi
        r2 = r2_score(y_test_kwh, pred_kwh)
        rmse = np.sqrt(mean_squared_error(y_test_kwh, pred_kwh))
        plt.subplot(1, 3, i + 1)
        plt.scatter(y_test_kwh, pred_kwh, alpha=0.6, edgecolors='k', color=colors[i])
        plt.plot([y_test_kwh.min(), y_test_kwh.max()], [y_test_kwh.min(), y_test_kwh.max()], '--r', linewidth=2)
        plt.title(f'{model_name}\nR2: {r2:.2f} | RMSE: {rmse:.2f} kWh')
        plt.xlabel('Nilai Aktual (kWh)')
        plt.ylabel('Nilai Prediksi (kWh)')
        plt.grid(True)
    plt.suptitle(f'Scatter Plot: Prediksi vs Aktual - {plot_suffix}', fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(os.path.join(output_dir, f'scatter_plot_{plot_suffix}.png'))
    plt.close()

    # --- Grafik Waktu (Titik Acak) ---
    plot_df = pd.DataFrame({'Aktual (kWh)': y_test_kwh})
    for model_name, pred_kwh in predictions_kwh.items():
        plot_df[f'Prediksi {model_name} (kWh)'] = pred_kwh
    plot_df.sort_index(inplace=True)
    
    plt.figure(figsize=(20, 8))
    plt.plot(plot_df.index, plot_df['Aktual (kWh)'], label='Nilai Aktual', color='blue', marker='o', linestyle='None', markersize=5, alpha=0.8)
    plt.plot(plot_df.index, plot_df['Prediksi RandomForest (kWh)'], label='Prediksi RF', color='green', marker='x', linestyle='None', markersize=4, alpha=0.8)
    plt.plot(plot_df.index, plot_df['Prediksi GradientBoosting (kWh)'], label='Prediksi GB', color='red', marker='x', linestyle='None', markersize=4, alpha=0.8)
    plt.plot(plot_df.index, plot_df['Prediksi LSTM (kWh)'], label='Prediksi LSTM', color='orange', marker='x', linestyle='None', markersize=4, alpha=0.8)
    plt.title(f'Grafik Waktu: Prediksi vs Aktual - {plot_suffix}\n(Menampilkan titik data dari Test Set yang acak)', fontsize=16)
    plt.xlabel('Waktu'); plt.ylabel('Konsumsi Energi (kWh)')
    plt.legend(); plt.grid(True); plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'time_series_plot_{plot_suffix}.png'))
    plt.close()

def create_combined_heatmap(performance_data, title_suffix, output_dir):
    """Membuat dan menyimpan heatmap gabungan dari data kinerja model."""
    if not performance_data:
        print(f"Tidak ada data kinerja untuk membuat heatmap.")
        return
    df = pd.DataFrame(performance_data)
    
    # Membuat label yang lebih deskriptif untuk pivot table
    df['Label Perangkat'] = df['Gedung'] + ' - ' + df['Perangkat']

    try:
        # Mengurutkan label agar lebih teratur di heatmap
        df.sort_values(by=['Gedung', 'Label Perangkat'], inplace=True)
        
        rmse_pivot = df.pivot_table(index='Model', columns='Label Perangkat', values='RMSE')
        r2_pivot = df.pivot_table(index='Model', columns='Label Perangkat', values='R2')
    except Exception as e:
        print(f"Error saat membuat pivot table untuk {title_suffix}: {e}\nData: {df}")
        return
        
    # Menyesuaikan ukuran gambar berdasarkan jumlah perangkat
    num_devices = len(df['Label Perangkat'].unique())
    fig_width = max(18, num_devices * 1.5) # Lebar minimum 18, bertambah 1.5 inch per perangkat
    
    fig, axes = plt.subplots(2, 1, figsize=(fig_width, 14))
    fig.suptitle(f'Heatmap Kinerja Model - {title_suffix}', fontsize=20)
    
    sns.heatmap(rmse_pivot, annot=True, fmt=".2f", cmap="viridis_r", ax=axes[0], linewidths=.5)
    axes[0].set_title('RMSE - Lebih Rendah Lebih Baik', fontsize=16)
    axes[0].set_xlabel('')
    axes[0].set_ylabel('Model', fontsize=12)
    axes[0].tick_params(axis='x', rotation=45)

    sns.heatmap(r2_pivot, annot=True, fmt=".2f", cmap="viridis", ax=axes[1], linewidths=.5)
    axes[1].set_title('R2 Score - Lebih Tinggi Lebih Baik', fontsize=16)
    axes[1].set_xlabel('Gedung - Perangkat / Lokasi', fontsize=12)
    axes[1].set_ylabel('Model', fontsize=12)
    axes[1].tick_params(axis='x', rotation=45)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    heatmap_path = os.path.join(output_dir, f'heatmap_{title_suffix}.png')
    plt.savefig(heatmap_path, bbox_inches='tight')
    plt.close()
    print(f"\nHeatmap gabungan disimpan di: {heatmap_path}")

print("✅ Fungsi-fungsi pembantu berhasil didefinisikan.")

✅ Fungsi-fungsi pembantu berhasil didefinisikan.


In [17]:
# ==============================================================================
# @title 5. Proses Utama: Melatih Model untuk Setiap File Data
# ==============================================================================
# Ini adalah inti dari skrip. Sel ini akan melakukan loop melalui semua file
# CSV, melakukan seleksi fitur, melatih model, dan menyimpan hasilnya.

all_performance_data = [] # PEMBARUAN: Satu list untuk semua data kinerja
building_predictions_tracker = {} # Untuk menyimpan prediksi gabungan per gedung

for root, dirs, files in os.walk(SOURCE_DATA_DIR):
    if not dirs and not files and root == SOURCE_DATA_DIR:
        print(f"Folder '{SOURCE_DATA_DIR}' kosong. Silakan unggah data Anda."); break
        
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            print(f"\n{'='*50}\nMemproses file: {file_path}\n{'='*50}")
            
            try:
                df = pd.read_csv(file_path, index_col='id_time', parse_dates=True)
            except Exception as e:
                print(f"   Gagal membaca file. Error: {e}"); continue
            
            existing_cols = [col for col in RELEVANT_COLUMNS if col in df.columns]
            df_cleaned = df.reindex(columns=existing_cols).dropna(subset=[TARGET_VARIABLE])
            
            if df_cleaned.empty: print("   Data kosong setelah dibersihkan."); continue

            print("   - Membuat fitur historis dari Konsumsi Energi...")
            for lag in range(1, 4):
                df_cleaned[f'Konsumsi_Energi_Lag_{lag}'] = df_cleaned[TARGET_VARIABLE].shift(lag)
            df_cleaned.dropna(inplace=True)
            if df_cleaned.empty: print("   Data kosong setelah membuat fitur historis."); continue

            print("   - Menghitung matriks korelasi...")
            correlation_matrix = df_cleaned.corr()
            
            relative_path = os.path.relpath(root, SOURCE_DATA_DIR)
            device_output_dir = os.path.join(RESULTS_DIR, relative_path)
            os.makedirs(device_output_dir, exist_ok=True)
            
            plt.figure(figsize=(22, 18)); sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
            plt.title(f'Correlation Matrix - {os.path.basename(root)}', fontsize=16); plt.xticks(rotation=45, ha='right'); plt.yticks(rotation=0)
            plt.tight_layout(); plt.savefig(os.path.join(device_output_dir, 'correlation_matrix.png')); plt.close()
            print(f"   - Heatmap korelasi disimpan di: {device_output_dir}")

            correlations = correlation_matrix[TARGET_VARIABLE].abs()
            selected_features = correlations[correlations >= 0.5].index.tolist()
            
            features_for_model = [f for f in selected_features if f != TARGET_VARIABLE]
            
            print(f"   - Fitur terpilih dengan korelasi >= 0.5: {features_for_model}")
            if not features_for_model: print("   - Tidak ada fitur yang memenuhi ambang korelasi."); continue

            # --- PEMBARUAN: Filter data untuk menghilangkan nilai 0 sebelum split ---
            print(f"   - Jumlah data sebelum filter nilai 0: {len(df_cleaned)}")
            df_filtered = df_cleaned[df_cleaned[TARGET_VARIABLE] > 0]
            print(f"   - Jumlah data setelah filter nilai 0: {len(df_filtered)}")

            if df_filtered.empty:
                print("   - Tidak ada data non-nol untuk dilatih. Melewati file ini.")
                continue

            X = df_filtered[features_for_model]; y = df_filtered[TARGET_VARIABLE]
            if len(X) < 20: print(f"   Data tidak cukup untuk pelatihan."); continue

            X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
            X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

            if len(X_train) == 0 or len(X_test) == 0: print(f"   Data tidak cukup setelah di-split."); continue

            print(f"   - Ukuran Data: Latih={len(X_train)}, Validasi={len(X_val)}, Uji={len(X_test)} (Acak)")
            model_evaluations = train_and_evaluate_models(X_train, y_train, X_val, y_val, X_test, y_test)
            
            device_name = os.path.basename(root)
            building_name = relative_path.split(os.sep)[0]
            
            device_predictions = {name: data['predictions'] for name, data in model_evaluations.items()}
            create_prediction_plots(y_test, device_predictions, device_name, device_output_dir)
            print(f"   - Plot prediksi untuk perangkat '{device_name}' disimpan.")

            if building_name not in building_predictions_tracker:
                building_predictions_tracker[building_name] = {'y_true': [], 'preds': {'RandomForest': [], 'GradientBoosting': [], 'LSTM': []}}
            
            building_predictions_tracker[building_name]['y_true'].append(y_test)
            for model_name, preds in device_predictions.items():
                building_predictions_tracker[building_name]['preds'][model_name].append(preds)
            
            best_model_name, best_model_rmse, best_model_object = '', float('inf'), None
            for model_name, eval_data in model_evaluations.items():
                if eval_data['rmse'] < best_model_rmse:
                    best_model_rmse, best_model_name, best_model_object = eval_data['rmse'], model_name, eval_data['model']
                
                # PEMBARUAN: Membuat label deskriptif dan menyimpan ke satu list
                path_parts = relative_path.split(os.sep)
                if len(path_parts) > 2: # e.g., witel/Lantai1/AHU
                    descriptive_name = f"{path_parts[1]}_{path_parts[2]}"
                else: # e.g., witel/Chiller
                    descriptive_name = path_parts[1]
                
                all_performance_data.append({
                    'Gedung': building_name,
                    'Perangkat': descriptive_name,
                    'Model': model_name,
                    'RMSE': eval_data['rmse'],
                    'R2': eval_data['r2']
                })
            
            if best_model_object:
                model_filename = os.path.join(device_output_dir, f'model_terbaik_{best_model_name}.joblib')
                if best_model_name != 'LSTM': joblib.dump(best_model_object, model_filename)
                else: best_model_object.save(model_filename.replace('.joblib', '.h5'))
                print(f"   ==> Model terbaik ({best_model_name}) disimpan.")

print("\n\n✅ Proses pelatihan untuk semua file selesai.")


Memproses file: sumber_data/witel/lantai1/ahu/Cleaned_Cleaned_WITEL_AHU_L1_hasil_interpolasi_1_Jam.csv
   - Membuat fitur historis dari Konsumsi Energi...
   - Menghitung matriks korelasi...
   - Heatmap korelasi disimpan di: hasil_model/witel/lantai1/ahu
   - Fitur terpilih dengan korelasi >= 0.5: ['Evapotranspiration', 'Sunshine Duration', 'Konsumsi_Energi_Lag_1', 'Konsumsi_Energi_Lag_2', 'Konsumsi_Energi_Lag_3']
   - Jumlah data sebelum filter nilai 0: 6260
   - Jumlah data setelah filter nilai 0: 5696
   - Ukuran Data: Latih=3987, Validasi=854, Uji=855 (Acak)
   - Melatih Random Forest...
   - Melatih Gradient Boosting...
   - Melatih LSTM...
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
   - Plot prediksi untuk perangkat 'ahu' disimpan.
   ==> Model terbaik (RandomForest) disimpan.

Memproses file: sumber_data/witel/lantai1/sdp/Cleaned_Cleaned_WITEL_SDP_L1_hasil_interpolasi_1_Jam.csv
   - Membuat fitur historis dari Konsumsi Energi...
   - Menghitung matriks korelasi...
   - Heatmap kor

In [18]:
# ==============================================================================
# @title 6. Membuat Plot Gabungan dan Heatmap Final
# ==============================================================================
# Sel terakhir ini akan membuat visualisasi gabungan untuk setiap gedung.

print("\nMembuat plot gabungan dan heatmap kinerja...")
for building, data in building_predictions_tracker.items():
    print(f"\n--- Memproses Gedung: {building.upper()} ---")
    y_true_combined = pd.concat(data['y_true'])
    preds_combined = {model: np.concatenate(preds) for model, preds in data['preds'].items()}
    
    building_output_dir = os.path.join(RESULTS_DIR, building)
    os.makedirs(building_output_dir, exist_ok=True)
    
    create_prediction_plots(y_true_combined, preds_combined, f"Gabungan_{building.upper()}", building_output_dir)
    print(f"   - Plot prediksi gabungan untuk gedung '{building}' disimpan di: {building_output_dir}")

# PEMBARUAN: Membuat satu heatmap gabungan untuk semua data
if not all_performance_data:
    print("Tidak ada data kinerja yang dihasilkan. Heatmap tidak dapat dibuat.")
else:
    create_combined_heatmap(all_performance_data, "Gabungan_Semua_Gedung", RESULTS_DIR)

print("\n\n🏁 Proses Selesai. Semua hasil telah disimpan di folder 'hasil_model'.")



Membuat plot gabungan dan heatmap kinerja...

--- Memproses Gedung: WITEL ---
   - Plot prediksi gabungan untuk gedung 'witel' disimpan di: hasil_model/witel

Heatmap gabungan disimpan di: hasil_model/heatmap_Gabungan_Semua_Gedung.png


🏁 Proses Selesai. Semua hasil telah disimpan di folder 'hasil_model'.
